<a href="https://colab.research.google.com/github/Edvandro-Nogueira/TCC-Univesp/blob/main/Testes_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

data = pd.read_table('https://raw.githubusercontent.com/jghm-f/FACTCK.BR/master/FACTCKBR.tsv')
data

,URL,Author,datePublished,claimReviewed,reviewBody,title,ratingValue,bestRating,alternativeName
0,https://aosfatos.org/noticias/governo-bolsonar...,https:www.aosfatos.org,2019-07-22,Espaço dedicado para os eleitores do Bolsonaro...,Publicações que circulam nas redes sociais vej...,Governo Bolsonaro não suspendeu distribuição d...,1.0,5,falso
1,https://aosfatos.org/noticias/nao-e-miriam-lei...,https:www.aosfatos.org,2019-07-22,Vos apresento a funcionária protegida pela Glo...,Uma foto de um treinamento de defesa contra as...,Não é Miriam Leitão quem segura fuzil ao lado ...,1.0,5,falso
2,https://aosfatos.org/noticias/nao-e-verdade-qu...,https:www.aosfatos.org,2019-07-22,Você sabia que Benedita da Silva já foi embaix...,falsa a informação de que a deputada federal ...,Não é verdade que Benedita da Silva foi embaix...,1.0,5,falso
3,https://aosfatos.org/noticias/nao-e-de-mulher-...,https:www.aosfatos.org,2019-07-18,QUEM A MULHER QUE EMPURROU O PADRE MARCELO ROSSI,Modificada digitalmente para incluir o filtro ...,Não é de mulher que empurrou padre Marcelo Ros...,1.0,5,falso
4,https://aosfatos.org/noticias/e-falso-que-refo...,https:www.aosfatos.org,2019-07-17,As crueldades da Reforma da PrevidênciaPessoas...,Uma publicação que circula nas redes sociais a...,É falso que reforma tira de pessoas com defici...,1.0,5,falso
...,...,...,...,...,...,...,...,...,...
1308,https://piaui.folha.uol.com.br/lupa/2018/05/17...,https:piaui.folha.uol.com.brlupa,2018-05-17 15:23:40,Cármen Lúcia e Raquel Dogde acabam de pedir pr...,Empty,#Verificamos: PGR e Cármen Lúcia não pediram p...,4.0,6,Falso
1309,https://piaui.folha.uol.com.br/lupa/2018/05/16...,https:piaui.folha.uol.com.brlupa,2018-05-16 17:14:19,Marina assina carta em prol do casamento homos...,Empty,#Verificamos: Marina não assinou carta 'em pro...,4.0,6,Falso
1310,https://piaui.folha.uol.com.br/lupa/2018/05/16...,https:piaui.folha.uol.com.brlupa,2018-05-16 12:00:30,Tribunal divulga documento que prova inocência...,Empty,#Verificamos: decisão de tribunal do DF não 'p...,4.0,6,Falso
1311,https://piaui.folha.uol.com.br/lupa/2018/05/15...,https:piaui.folha.uol.com.brlupa,2018-05-15 13:00:20,Direção do Hospital das Clínicas de São Paulo ...,Empty,#Verificamos: vacina contra gripe não causa 's...,4.0,6,Falso


In [2]:
data = data.drop(['URL', 'Author', 'datePublished', 'claimReviewed', 'reviewBody', 'ratingValue', 'bestRating'], axis=1)
data = data.reindex(columns=['alternativeName', 'title'])
data.columns = ['label', 'preprocessed_news']
data = data[(data['label'] == 'verdadeiro') | (data['label'] == 'Verdadeiro') | (data['label'] == 'falso') | (data['label'] == 'Falso')]
data['label'] = data['label'].map({'verdadeiro': 1, 'Verdadeiro': 1, 'Falso': 0, 'falso': 0})
data['preprocessed_news'] = data['preprocessed_news'].str.replace('#Verificamos:', '')
data['preprocessed_news'] = data['preprocessed_news'].str.replace('#VerificamosENEM:', '')
#VerificamosENEM:
data

,label,preprocessed_news
0,0,Governo Bolsonaro não suspendeu distribuição d...
1,0,Não é Miriam Leitão quem segura fuzil ao lado ...
2,0,Não é verdade que Benedita da Silva foi embaix...
3,0,Não é de mulher que empurrou padre Marcelo Ros...
4,0,É falso que reforma tira de pessoas com defici...
...,...,...
1308,0,PGR e Cármen Lúcia não pediram prisão imediat...
1309,0,Marina não assinou carta 'em prol do casament...
1310,0,decisão de tribunal do DF não 'prova inocênci...
1311,0,vacina contra gripe não causa 'surto mortal' ...


In [3]:
!pip install nltk

In [4]:
import pandas as pd
import gensim
from gensim import corpora
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Definir a lista de stopwords
stop_words = stopwords.words('portuguese')

# Converter a coluna 'preprocessed_news' em uma lista de listas, removendo as stopwords
corpus = [[word for word in doc.split() if word not in stop_words] for doc in data['preprocessed_news']]

# Criar um dicionário a partir do corpus
dictionary = corpora.Dictionary(corpus)

# Remover palavras muito raras ou muito comuns do dicionário
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Criar uma matriz de termos de documento (DTM)
corpus = [dictionary.doc2bow(doc) for doc in corpus]

# Treinar um modelo LDA com 5 tópicos
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=dictionary,
                                            num_topics=5,
                                            random_state=42,
                                            passes=50)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Atribuir tópicos a cada documento no corpus
topics = lda_model[corpus]

# Criar um dicionário para mapear os IDs dos tópicos aos nomes dos tópicos
topic_names = {0: 'Política', 1: 'Esportes', 2: 'Negócios', 3: 'Entretenimento', 4: 'Saúde'}

# Selecionar alguns documentos de cada tópico e imprimir o texto correspondente
for topic_id, topic in lda_model.show_topics(num_topics=5, formatted=False):
    print('Tópico:', topic_names[topic_id])
    print('Palavras-chave:', ', '.join([word[0] for word in topic]))
    print('Documentos:')
    for doc in sorted(lda_model.get_topic_terms(topic_id), key=lambda x: -x[1])[:5]:
        if doc[1] > 0.5:
            print(data.iloc[doc[0]]["preprocessed_news"])
    print()

Tópico: Política
Palavras-chave: -, Agência, Pública, sobre, Checamos, dados, frases, usa, falsos, contra
Documentos:

Tópico: Esportes
Palavras-chave: falso, Bolsonaro, Vídeo, PT, candidatos, preso, Gleisi, antiga, principais, cocaína
Documentos:

Tópico: Negócios
Palavras-chave: |, Fatos, Aos, Bolsonaro, Não, foto, mostra, verdade, falsa, Foto
Documentos:

Tópico: Entretenimento
Palavras-chave: falso, Lula, Bolsonaro, contra, R$, Marielle, foto, Brasil, falsa, Dilma
Documentos:

Tópico: Saúde
Palavras-chave: |, Fatos, Aos, falso, disse, É, Haddad, vídeo, informações, redes
Documentos:



In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Definir a lista de stopwords
stop_words = stopwords.words('portuguese')

# Converter a coluna 'preprocessed_news' em uma lista de documentos
docs = data['preprocessed_news'].tolist()

# Criar uma matriz de frequência de termos (TF-IDF)
vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=5, max_df=0.5)
tfidf = vectorizer.fit_transform(docs)

# Treinar um modelo NMF com 5 tópicos
nmf_model = NMF(n_components=5, random_state=42)
nmf_model.fit(tfidf)

# Extrair os tópicos e as palavras mais importantes de cada tópico
feature_names = vectorizer.get_feature_names_out()
topics = []
for topic_idx, topic in enumerate(nmf_model.components_):
    top_features_idx = topic.argsort()[:-11:-1]
    top_features = [feature_names[i] for i in top_features_idx]
    topics.append(' '.join(top_features))

# Categorizar os documentos de acordo com o tópico mais relevante
doc_topic = nmf_model.transform(tfidf)
data['topic'] = np.argmax(doc_topic, axis=1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Imprimir as amostras dos documentos em cada tópico
for topic_idx, topic in enumerate(topics):
    print(f'Tópico {topic_idx}:')
    docs_in_topic = data[data['topic'] == topic_idx]['preprocessed_news'].sample(5, random_state=42)
    for doc in docs_in_topic:
        print(doc)
    print('\n')

Tópico 0:
  falso que governador do Maranhão substituiu bandeira nacional pela do PCdoB
  falso perfil de Maria do Rosário que considera corrupção algo cultural no Brasil
  falso que alunos de universidade federal tenham criado ateliê de arte anal
  falso que bula confirme relação de vacina com autismo
  falso que o Brasil comprou ou ganhou de Israel 15 helicópteros militares


Tópico 1:
Bolsonaro usa informação falsa ao criticar plano de governo do PT - Agência Pública
Dados falsos e omissões marcam Estatuto do Armamento - Agência Pública
Corrente erra ao considerar ilegal gravação de Temer sem autorização de juiz - Agência Pública
Ciro Gomes exagera dados sobre trabalho, impostos e segurança - Agência Pública
Checamos 8 frases da votação da denúncia contra Temer - Agência Pública


Tópico 2:
Juiz Sérgio Moro não está com doença grave; notícia é falsa | Aos Fatos
André Esteves não é o dono da revista Veja | Aos Fatos
É falso que Dilma gastou R$ 73 milhões em salão de beleza | Aos Fato

In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Definir a lista de stopwords
stop_words = stopwords.words('portuguese')

# Converter a coluna 'preprocessed_news' em uma lista de documentos
docs = data['preprocessed_news'].tolist()

# Criar uma matriz de frequência de termos (TF-IDF)
vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=5, max_df=0.5)
tfidf = vectorizer.fit_transform(docs)

# Treinar um modelo NMF com 5 tópicos
nmf_model = NMF(n_components=5, random_state=42)
nmf_model.fit(tfidf)

# Extrair os tópicos e as palavras mais importantes de cada tópico
feature_names = vectorizer.get_feature_names_out()
topics = []
for topic_idx, topic in enumerate(nmf_model.components_):
    top_features_idx = topic.argsort()[:-11:-1]
    top_features = [feature_names[i] for i in top_features_idx]
    topics.append(' '.join(top_features))

# Mapear os índices de tópicos extraídos para os nomes de tópicos desejados
topic_names = {
    0: 'Política',
    1: 'Esportes',
    2: 'Negócios',
    3: 'Entretenimento',
    4: 'Saúde'
}

# Categorizar os documentos de acordo com o tópico mais relevante
doc_topic = nmf_model.transform(tfidf)
data['topic'] = np.argmax(doc_topic, axis=1)

# Imprimir as amostras dos documentos em cada tópico
for topic_idx, topic in enumerate(topics):
    topic_name = topic_names[topic_idx]
    print(f'Tópico {topic_name}:')
    docs_in_topic = data[data['topic'] == topic_idx]['preprocessed_news'].sample(5, random_state=42)
    for doc in docs_in_topic:
        print(doc)
    print('\n')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Tópico Política:
  falso que governador do Maranhão substituiu bandeira nacional pela do PCdoB
  falso perfil de Maria do Rosário que considera corrupção algo cultural no Brasil
  falso que alunos de universidade federal tenham criado ateliê de arte anal
  falso que bula confirme relação de vacina com autismo
  falso que o Brasil comprou ou ganhou de Israel 15 helicópteros militares


Tópico Esportes:
Bolsonaro usa informação falsa ao criticar plano de governo do PT - Agência Pública
Dados falsos e omissões marcam Estatuto do Armamento - Agência Pública
Corrente erra ao considerar ilegal gravação de Temer sem autorização de juiz - Agência Pública
Ciro Gomes exagera dados sobre trabalho, impostos e segurança - Agência Pública
Checamos 8 frases da votação da denúncia contra Temer - Agência Pública


Tópico Negócios:
Juiz Sérgio Moro não está com doença grave; notícia é falsa | Aos Fatos
André Esteves não é o dono da revista Veja | Aos Fatos
É falso que Dilma gastou R$ 73 milhões em salão

In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Definir a lista de stopwords
stop_words = stopwords.words('portuguese')

# Converter a coluna 'preprocessed_news' em uma lista de documentos
docs = data['preprocessed_news'].tolist()

# Criar uma matriz de frequência de termos (TF-IDF)
vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=5, max_df=0.5)
tfidf = vectorizer.fit_transform(docs)

# Categorizar os documentos de acordo com o tópico mais relevante usando Naive Bayes Multinomial
nb_model = MultinomialNB()
nb_model.fit(tfidf, data['topic'])

# Imprimir as amostras dos documentos em cada tópico
for topic_name in ['Política', 'Esportes', 'Negócios', 'Entretenimento', 'Saúde']:
    print(f'Tópico {topic_name}:')
    docs_in_topic = data[data['topic'] == topic_name]['preprocessed_news']
    if len(docs_in_topic) >= 5:
        docs_sample = docs_in_topic.sample(5, random_state=42)
        for doc in docs_sample:
            print(doc)
    else:
        print('Não há documentos suficientes para este tópico.')
    print('\n')

Tópico Política:
Não há documentos suficientes para este tópico.


Tópico Esportes:
Não há documentos suficientes para este tópico.


Tópico Negócios:
Não há documentos suficientes para este tópico.


Tópico Entretenimento:
Não há documentos suficientes para este tópico.


Tópico Saúde:
Não há documentos suficientes para este tópico.




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
